# Lista Aberta do Módulo 3 - Comparando Regressão Lasso e KNN

**Regras para envio:**

1. Faça uma cópia deste arquivo
2. Deixe o arquivo aberto para qualquer pessoa com o link (Share > Anyone with the link)
3. Garanta que as saídas do notebook estão salvas (Settings > deixe desligada a opção (New notebooks use private outputs (omit outputs when saving)), e acesse o link depois em outro navegador para garantir que estão!)
4. Digite seu nome: Arthur Pontes Nader
5. Após finalizar o exercício, poste o link (apenas o link!) no moodle.

**Atenção**: como temos pouco tempo para o fechamento do semestre, por favor garantam que seus links estão corretos! Não teremos tempo para correções. É essencial que os alunos de ICD saibam usar o Google Colab.

## Predizendo a glicose do sangue - Diabetes Dataset

Neste trabalho você deverá realizar uma **comparação entre o k-Nearest Neighbors (KNN) e a Regressão Lasso para predição da glicose do sangue**. Para isso, usaremos o dataset de diabetes de Stanford (já embutido no sklearn). Notem que as variáveis já estão normalizadas.

A avaliação dos métodos deverá ser feito usando **o erro quadrado médio, usando validação cruzada k-fold com k igual a 5**.  

Você não precisa implementar os métodos, já que estão disponíveis na biblioteca scikit-learn da linguagem Python. Se necessário, pode fazer mais importações de bibliotecas.

## Importando módulos e baixando dataset

In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV

In [ ]:
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data,columns =[diabetes.feature_names])

In [ ]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [ ]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [ ]:
# Variavel que queremos predizer
TARGET = "s6"

# Número de folds da validação cruzada
SPLITS = 5

Se tiver dúvidas sobre a validação cruzada, recomendo :https://machinelearningmastery.com/k-fold-cross-validation/ . Busque também outros materiais, o importante é você conhecer e saber utilizar a validação cruzada, que é bem mais robusta que a simples divisão em treino e teste.

# Exercício 01:

Execute a regressão lasso para explicar o fator s6 (isto é, glicose) dos dados a partir dos outros fatores.

Para a questão, faça as seguintes tarefas:

* Realize validação cruzada, com k-fold igual a 5
* Reporte o erro quadrado médio (dos 5 folds de validação do cross validation)

In [ ]:
# YOUR CODE HERE

def validacaoCruzada(dataset, k, alvo):

  #Variavel alvo
  y = dataset[alvo].values

  #Restante dos dados
  colunas = list(df.columns)
  colunas.remove((TARGET,))
  x = dataset[colunas].values

  #Criacao do modelo classificador
  modelo = Lasso()

  #Criacao do KFold
  kfold = KFold(n_splits=k, shuffle=True, random_state=1007)

  #Lista que ira guardar os resultados obtidos
  resultados = []

  #Geracao dos resultados
  for treino, teste in kfold.split(x, y):
      x_treino, x_teste = x[treino], x[teste]
      y_treino, y_teste = y[treino], y[teste]
      modelo.fit(x_treino, y_treino)

      #avaliacao do modelo
      predicao = modelo.predict(x_teste)
      erro_quadrado_medio = metrics.mean_squared_error(y_teste, predicao)
      resultados.append(erro_quadrado_medio)

  # Exibicao dos resultados obtidos
  for cnt in range(1, k+1):
    print("Erro Quadrado Medio %d: %.5f" %(cnt, resultados[cnt-1]))

  print("\nMedia: %.5f" %np.mean(resultados))

  return None

In [ ]:
validacaoCruzada(df, SPLITS, TARGET)

Erro Quadrado Medio 1: 0.00196
Erro Quadrado Medio 2: 0.00207
Erro Quadrado Medio 3: 0.00236
Erro Quadrado Medio 4: 0.00215
Erro Quadrado Medio 5: 0.00282

Media: 0.00227


# Exercício 02:

Execute a knn para explicar o fator s6 (isto é, glicose) dos dados a partir dos outros fatores.

Para a questão, faça as seguintes tarefas:

* Escolha e reporte resultados com 3 números de vizinhos diferentes (k)
* Realize validação cruzada, com k-fold igual a 5 (para cada um dos k vizinhos)
* Reporte o erro quadrado médio (dos 5 folds de validação do cross validation)

OBS: Não confundir o k-fold com os k vizinhos! Um conceito não tem relação com o outro.

In [ ]:
def execucaoKNN(dataset, alvo, k, k_neigh):

  #Variavel alvo
  y = dataset[alvo].values

  #Restante dos dados
  colunas = list(df.columns)
  colunas.remove((TARGET,))
  x = dataset[colunas].values

  #Criacao do modelo classificador
  modelo = KNeighborsRegressor(n_neighbors=k_neigh)

  #Criacao do KFold
  kfold = KFold(n_splits=k, shuffle=True, random_state=1007)

  #Lista que ira guardar os resultados obtidos
  resultados = []

  #Geracao dos resultados
  for treino, teste in kfold.split(x, y):
      x_treino, x_teste = x[treino], x[teste]
      y_treino, y_teste = y[treino], y[teste]
      modelo.fit(x_treino, y_treino)

      #avaliacao do modelo
      predicao = modelo.predict(x_teste)
      erro_quadrado_medio = metrics.mean_squared_error(y_teste, predicao)
      resultados.append(erro_quadrado_medio)


  # Exibicao dos resultados obtidos
  for cnt in range(1, k+1):
    print("Erro Quadrado Medio %d: %.5f" %(cnt, resultados[cnt-1]))

  print("\nMedia: %.5f" %np.mean(resultados))

  return None

In [ ]:
print("Execucao para 3 vizinhos:")
execucaoKNN(df, TARGET, SPLITS, 3)
print("\nExecucao para 4 vizinhos:")
execucaoKNN(df, TARGET, SPLITS, 4)
print("\nExecucao para 5 vizinhos:")
execucaoKNN(df, TARGET, SPLITS, 5)

Execucao para 3 vizinhos:
Erro Quadrado Medio 1: 0.00212
Erro Quadrado Medio 2: 0.00190
Erro Quadrado Medio 3: 0.00250
Erro Quadrado Medio 4: 0.00230
Erro Quadrado Medio 5: 0.00242

Media: 0.00225

Execucao para 4 vizinhos:
Erro Quadrado Medio 1: 0.00205
Erro Quadrado Medio 2: 0.00176
Erro Quadrado Medio 3: 0.00239
Erro Quadrado Medio 4: 0.00220
Erro Quadrado Medio 5: 0.00221

Media: 0.00212

Execucao para 5 vizinhos:
Erro Quadrado Medio 1: 0.00192
Erro Quadrado Medio 2: 0.00165
Erro Quadrado Medio 3: 0.00226
Erro Quadrado Medio 4: 0.00213
Erro Quadrado Medio 5: 0.00222

Media: 0.00204


# Exercício 03:

Compare os resultados (Pode ser bem direto ao ponto).

Pela comparação dos resultados da regressão lasso e dos k-vizinhos, pode-se perceber que a média dos erros na regressão lasso é bem próxima da média obtida na execução para 3 vizinhos.<br>
Além disso, analisando-se os resultados das 3 execuções de vizinhos próximos, percebe-se que há uma tendência de diminuição no erro quadrado médio à medida que o número de vizinhos aumenta.